In [1]:
import numpy as np
import scipy as sp
import pywt
import math
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture
from sklearn.cluster import AffinityPropagation

In [ ]:
#Read in waveform and perform Haar wavelet decomposition
wfbase = np.load("g032_waveforms_phys.npy")
result = pywt.wavedec(wfbase,'haar',level=3)
print(len(result))

In [ ]:
#Look at the effect of waveform reconstruction (inverse Haar wavelet transform)
wf_display = wfbase[23]
result2 = pywt.dwt(wf_display,'haar')
wf_recon = pywt.idwt(result2[0],result2[1], 'haar')
plt.plot(wf_display, label="Original Waveform")
plt.plot(wf_recon,linestyle="-", label="Reconstructed Waveform")
plt.legend()

In [ ]:
#Look at the effect of waveform reconstruction with or without the detailed coefficient
plt.plot(pywt.idwt(result2[0],None, 'haar'), label="No Detailed Coefficient")
plt.plot(pywt.idwt(result2[0],result2[1], 'haar'),alpha=0.5, label="With Detailed Coefficient")
plt.legend()
plt.xlim(0,100)
plt.ylim(-0.06,0.06)

In [ ]:
# Make t-SNE plot of all coefficients (approximate and detailed) to look at their distributions
plt.figure(figsize=(20,20))
for i in range(len(result)):
    pindex = 331 + i
    plt.subplot(pindex)
    X_embedded = TSNE(n_components=2).fit_transform(result[i][:1000])
    if i == 0:
        lb = "Approximation Coefficient"
    else:
        lb = "Detailed Coefficient Level %d"%(i)
    plt.scatter(X_embedded[:,0], X_embedded[:,1], label=lb, color="red")
    plt.legend()

In [ ]:
#Load the waveform again
wfbase = np.load("g032_waveforms_phys.npy")
result = pywt.wavedec(wfbase,'haar',level=3)

In [ ]:
#Apply Affinity Propagation to the approximate coefficient of HaarDWT result

# normalizing waveform with a scipy scaler: commented out because it doesn't perform very well
# from sklearn.preprocessing import StandardScaler, RobustScaler
# scaled_ac = StandardScaler().fit_transform(result[0])

plt.figure(figsize=(30,30))
ap = AffinityPropagation(max_iter=500).fit(X=result[0])
nwave = len(ap.cluster_centers_indices_)
print(nwave)
for j in range(len(ap.cluster_centers_indices_)):
    plt.subplot(math.ceil((nwave)**0.5),math.ceil((nwave)**0.5),j+1)
    #min_index = np.argmin(np.linalg.norm(result[0] - np.repeat(X[j].reshape(1,-1), result[0].shape[0], axis=0),ord=2,axis=1))
    coeffl = [res[ap.cluster_centers_indices_[j]] for res in result]
    #coeffl[0] = X[j]
    wf_recon = pywt.waverec(coeffl, 'haar')
    plt.plot(wf_recon,linestyle="-", label="Reconstructed Waveform")
plt.savefig("exemplar.png")